In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train = pd.read_csv('./data/Train_Fyxd0t8.csv', index_col='ID')
test = pd.read_csv('./data/Test_C1XBIYq.csv', index_col='ID')

In [44]:
%run scripts/features.py
%run scripts/eval.py

In [4]:
# impute missing values based on Pesticide_Use_Category

# mean_per_pesticide_use_category_train = train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].mean()

# def impute_values_train(row):
#     if pd.isnull(row.Number_Weeks_Used):
#         return mean_per_pesticide_use_category_train.ix[row.Pesticide_Use_Category]
#     else:
#         return row.Number_Weeks_Used

# train['Number_Weeks_Used'] = train.apply(impute_values_train, axis=1)
median_num_weeks_used_train = int(train.Number_Weeks_Used.median())
median_num_weeks_used_test = int(test.Number_Weeks_Used.median())

train = train.fillna(median_num_weeks_used_train)
test = test.fillna(median_num_weeks_used_test)

In [5]:
# mean_per_pesticide_use_category_test = test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].mean()

# def impute_values_test(row):
#     if pd.isnull(row.Number_Weeks_Used):
#         return mean_per_pesticide_use_category_test.ix[row.Pesticide_Use_Category]
#     else:
#         return row.Number_Weeks_Used

# test['Number_Weeks_Used'] = test.apply(impute_values_test, axis=1)

In [6]:
y = train.Crop_Damage

In [7]:
from sklearn.cross_validation import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=44)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import xgboost as xgb

In [45]:
ft = FeatureTransformer()
# scaler = MinMaxScaler()
# clf = LogisticRegression(C=0.001)
# clf = KNeighborsClassifier()
# clf = RandomForestClassifier()
clf = xgb.XGBClassifier()

pipeline = Pipeline([('ft', ft), ('clf', clf)])

In [38]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [39]:
# cross validation
scores = eval_models([pipeline], X_train, y_train)

accuracy score: 0.837319
combined score: 0.837319
accuracy score: 0.837319
combined score: 0.837319
accuracy score: 0.840273
combined score: 0.840273
accuracy score: 0.839218
combined score: 0.839218
accuracy score: 0.836545
combined score: 0.836545


In [40]:
print 'Mean Scores %f ' %(np.mean(scores))

Mean Scores 0.838135 


In [41]:
predsTest = pipeline.predict(X_test)
print 'Classification report %s ' %(classification_report(y_test, predsTest))

Classification report              precision    recall  f1-score   support

          0       0.84      1.00      0.91     14837
          1       0.64      0.04      0.07      2457
          2       0.00      0.00      0.00       478

avg / total       0.79      0.84      0.77     17772
 


In [42]:
confusion_matrix(y_test, predsTest)

array([[14817,    20,     0],
       [ 2363,    94,     0],
       [  445,    33,     0]])

In [43]:
# fit on the whole dataset
pipeline.fit(train, y)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [64]:
predictions = pipeline.predict(test)

In [65]:
submissions = pd.read_csv('./data/Sample_Submission_Psj3sjG.csv')

In [66]:
submissions['ID'] = test.index.values
submissions['Crop_Damage'] = predictions

In [67]:
submissions.to_csv('./submissions/tenth.csv', index=False)